In [ ]:
from sklearn.metrics import mean_squared_error

from sklearn.metrics import mean_absolute_error as MAE

from sklearn.preprocessing import OneHotEncoder

from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import TimeSeriesSplit
import xgboost as xgb
from sklearn.preprocessing import StandardScaler,MaxAbsScaler,MinMaxScaler,OrdinalEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import RegressorChain
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
%pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 1.3 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd "drive/MyDrive/Colab Notebooks/Challenge/"

/content/drive/MyDrive/Colab Notebooks/Challenge


In [ ]:

train_station = pd.read_csv("data_ewm1.2.csv", sep=",")
train_station['date'] = pd.to_datetime(train_station['date'])
train_station['Postcode'] = train_station['Postcode'].astype(str)
state="Charging"
df7=train_station[["date" ,"Station",state]].pivot(index='date', columns='Station', values=state).copy()
index_rem=df7[df7.eq(df7.iloc[:, 5], axis=0).all(1)].index
index_rem
train_station=train_station[train_station.date.isin(index_rem)==False].copy()

train_station=train_station[((train_station[["Available","Charging","Passive","Other"]].sum(axis=1)>3)|(
    train_station[["Available","Charging","Passive","Other"]].sum(axis=1)<3))==False].reset_index(drop=1).copy()



train_station = train_station.groupby(['date', 'area']).agg({'Available': 'sum',
                                                         'Charging': 'sum',
                                                          'Passive': 'sum',
                                                          'Other': 'sum',
                                                          'tod': 'max',
                                                          'dow': 'max',
                                                          'Latitude': 'mean',
                                                          'Longitude': 'mean',
                                                          'trend': 'max'}).reset_index()

#val 
station_val=pd.read_csv("train.csv", sep=",")
station_val['date'] = pd.to_datetime(station_val['date'])
station_val['Postcode'] = station_val['Postcode'].astype(str)


station_val = station_val.groupby(['date', 'area']).agg({'Available': 'sum',
                                                         'Charging': 'sum',
                                                          'Passive': 'sum',
                                                          'Other': 'sum',
                                                          'tod': 'max',
                                                          'dow': 'max',
                                                          'Latitude': 'mean',
                                                          'Longitude': 'mean',
                                                          'trend': 'max'}).reset_index()
#test
test_station=pd.read_csv("test.csv", sep=",")
test_station['date'] = pd.to_datetime(test_station['date'])
test_station['Postcode'] = test_station['Postcode'].astype(str)
test_station = test_station.groupby(['date', 'area']).agg({
                                                          'tod': 'max',
                                                          'dow': 'max',
                                                          'Latitude': 'mean',
                                                          'Longitude': 'mean',
                                                          'trend': 'max'}).reset_index()

In [ ]:


# add data
train_station['hour'] = train_station.date.dt.hour
train_station['min'] = train_station.date.dt.minute
train_station['quarter'] = train_station.date.dt.quarter
train_station['dayofmonth'] = train_station.date.dt.day
test_station['hour'] = test_station.date.dt.hour
test_station['min'] = test_station.date.dt.minute
test_station['dayofmonth'] = test_station.date.dt.day


df_train=train_station.copy().loc[train_station.date < '2021-02-1 00:00:00']
df_val=station_val.copy().loc[station_val.date >= '2021-02-1 00:00:00']
df_val['hour'] = df_val.date.dt.hour
df_val['min'] = df_val.date.dt.minute
df_val['quarter'] = df_val.date.dt.quarter
df_val['dayofmonth'] = df_val.date.dt.day

In [ ]:
col_act=["hour","tod","min","dow","trend"]
targets=["Available","Charging"	,"Passive",	"Other"	]
cat_x_test=df_val[col_act].copy()
cat_y_test=df_val[targets].copy()

In [ ]:


from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputRegressor
from sklearn.pipeline import Pipeline
import catboost as cb
cat_features_index = [#0,1,3
                      ]
col_act=["min","trend"]
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(train_station[["hour","tod","dow"]])
X=train_station.copy()
nameHot=enc.get_feature_names_out()
X[nameHot]=enc.transform(X[["hour","tod","dow"]]).toarray()


params = {'depth': [ 2,3,4,5],
          'learning_rate' : [0.01, 0.05,.1],
         'l2_leaf_reg': [1,2],
         'iterations': [500,1000]}
cbb = cb.CatBoostRegressor(loss_function="MultiRMSE",eval_metric="MultiRMSE")

#cb_model = GridSearchCV(cbb, params, scoring="neg_root_mean_squared_error"
#, cv = TimeSeriesSplit(n_splits=3).get_n_splits([cat_x_train,cat_y_train]),n_jobs=-1,verbose=1,return_train_score=True)
#cb_model.fit(cat_x_train,cat_y_train,verbose=200)
dec_values={'depth': 3, 'iterations': 1000, 'l2_leaf_reg': 1, 'learning_rate': .1}
#print(cb_model.best_params_,cb_model.best_score_)
models={}
for ar in train_station.area.unique():
  cat_x_train= X.loc[X.area==ar,col_act+list(nameHot)].copy()
  cat_y_train=X.loc[X.area==ar,targets].copy()
  clf = cb.CatBoostRegressor(eval_metric="MultiRMSE",loss_function="MultiRMSE",one_hot_max_size=200,depth= dec_values["depth"],
                            iterations = dec_values["iterations"], learning_rate= dec_values["learning_rate"],
                            l2_leaf_reg= dec_values["l2_leaf_reg"])# 0.1
  chain=RegressorChain(clf,order=[0,1,2,3]).fit(cat_x_train,cat_y_train, cat_features= cat_features_index,verbose=100)
  models[ar]=chain

# {'depth': 5, 'iterations': 1000, 'l2_leaf_reg': 1, 'learning_rate': 0.01}

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3678: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self[col] = igetitem(value, i)


0:	learn: 7.7336246	total: 2.36ms	remaining: 2.36s
100:	learn: 4.1426541	total: 214ms	remaining: 1.9s
200:	learn: 3.6851443	total: 440ms	remaining: 1.75s
300:	learn: 3.4676713	total: 650ms	remaining: 1.51s
400:	learn: 3.3457475	total: 866ms	remaining: 1.29s
500:	learn: 3.2742657	total: 1.07s	remaining: 1.07s
600:	learn: 3.2211333	total: 1.29s	remaining: 860ms
700:	learn: 3.1811108	total: 1.5s	remaining: 638ms
800:	learn: 3.1442290	total: 1.7s	remaining: 423ms
900:	learn: 3.1149401	total: 1.91s	remaining: 210ms
999:	learn: 3.0930713	total: 2.13s	remaining: 0us
0:	learn: 2.8381179	total: 2.7ms	remaining: 2.69s
100:	learn: 2.3191612	total: 218ms	remaining: 1.94s
200:	learn: 2.1665038	total: 433ms	remaining: 1.72s
300:	learn: 2.0834681	total: 645ms	remaining: 1.5s
400:	learn: 2.0289875	total: 871ms	remaining: 1.3s
500:	learn: 1.9843282	total: 1.08s	remaining: 1.08s
600:	learn: 1.9465424	total: 1.33s	remaining: 880ms
700:	learn: 1.9174911	total: 1.56s	remaining: 668ms
800:	learn: 1.8908833	

In [ ]:
df_val[nameHot]=enc.transform(df_val[["hour","tod","dow"]]).toarray()

for ar in train_station.area.unique():
  cat_x_test=df_val.loc[df_val.area==ar,col_act+list(nameHot)].copy()
  cat_y_test=df_val.loc[df_val.area==ar,targets].copy()
  y_pred_class=np.floor(#cb_model.best_estimator_.predict(cat_x_test)).astype(int)
                      models[ar].predict(cat_x_test)).astype(int)

  df55=cat_y_test.copy()
  df55[[i+"_" for i in targets]]=y_pred_class



  mea00=MAE(df55[[i+"_" for i in targets]],df55[targets],multioutput="raw_values")
  print("we do ",to_find)
  print(f'sklearn MAE \n{mea00}\n')
  print(f'\n{mea00.mean()}\n\n')
  print(f'sklearn MAE on the mean \n\n{100*mea00/df55[targets].mean(axis=0)}%')
#[5.47350898 2.10856977 1.41632889 8.37072959]

#sklearn MAE on the mean 
# 6.50347 #4.829137731481481
#sklearn MAE on the mean  #6.051359953703704


#[17.98726852  4.78472222  3.45023148 29.07002315]
#13.823061342592592 13.302083333333332 #6.686921296296297 12.660
#sklearn MAE on the mean 
#Available    13.727586
#Charging     18.405236
#Passive      21.208025
#Other        42.381061
#3.87108922363847  2.2569444444444446.  2.379345307068366. 4.323206018518519

we do  Available
sklearn MAE 
[3.0764774  1.59849363 1.15295481 9.67497103]


3.875724217844727


sklearn MAE on the mean 

Available     8.767006
Charging     26.875122
Passive      35.624776
Other        48.086503
dtype: float64%
we do  Available
sklearn MAE 
[2.45138889 1.38368056 1.22511574 3.87673611]


2.234230324074074


sklearn MAE on the mean 

Available     6.799795
Charging     26.853100
Passive      24.004989
Other        24.841473
dtype: float64%
we do  Available
sklearn MAE 
[2.34704519 1.68655852 1.0608343  4.49594438]


2.397595596755504


sklearn MAE on the mean 

Available     6.091546
Charging     22.641363
Passive      35.327031
Other        46.469848
dtype: float64%
we do  Available
sklearn MAE 
[ 2.16724537  1.63310185  1.26736111 11.31886574]


4.096643518518519


sklearn MAE on the mean 

Available    10.106871
Charging     21.884451
Passive      25.692163
Other        48.734240
dtype: float64%


In [ ]:
allfortest=[]
df_test=test_station.copy()
df_test[nameHot]=enc.transform(df_test[["hour","tod","dow"]]).toarray()

for ar in test_station.area.unique():
  df333=df_test[test_station.area==ar].copy()
  test_pred_class=(models[ar].predict(df333[col_act+list(nameHot)])).astype(int)
  df333[targets]=test_pred_class
  allfortest.append(df333)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3678: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self[col] = igetitem(value, i)


In [ ]:
pdDF=pd.concat(allfortest)
df335=test_station.copy()
test_merge=pd.merge(df335,pdDF,how="left",on=["date","area"])
test_merge

,date,area,tod_x,dow_x,Latitude_x,Longitude_x,trend_x,hour_x,min_x,dayofmonth_x,...,dow_2,dow_3,dow_4,dow_5,dow_6,dow_7,Available,Charging,Passive,Other
0,2021-02-19 00:00:00,east,0,6,48.865007,2.386981,22203,0,0,19,...,0.0,0.0,0.0,0.0,1.0,0.0,35,7,2,29
1,2021-02-19 00:00:00,north,0,6,48.878152,2.327133,22203,0,0,19,...,0.0,0.0,0.0,0.0,1.0,0.0,35,3,3,22
2,2021-02-19 00:00:00,south,0,6,48.838118,2.347826,22203,0,0,19,...,0.0,0.0,0.0,0.0,1.0,0.0,39,5,3,14
3,2021-02-19 00:00:00,west,0,6,48.851190,2.294069,22203,0,0,19,...,0.0,0.0,0.0,0.0,1.0,0.0,25,5,3,32
4,2021-02-19 00:15:00,east,1,6,48.865007,2.386981,22204,0,15,19,...,0.0,0.0,0.0,0.0,1.0,0.0,35,7,2,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7291,2021-03-10 23:30:00,west,94,4,48.851190,2.294069,24121,23,30,10,...,0.0,0.0,1.0,0.0,0.0,0.0,25,7,3,31
7292,2021-03-10 23:45:00,east,95,4,48.865007,2.386981,24122,23,45,10,...,0.0,0.0,1.0,0.0,0.0,0.0,37,5,1,30
7293,2021-03-10 23:45:00,north,95,4,48.878152,2.327133,24122,23,45,10,...,0.0,0.0,1.0,0.0,0.0,0.0,34,5,4,21
7294,2021-03-10 23:45:00,south,95,4,48.838118,2.347826,24122,23,45,10,...,0.0,0.0,1.0,0.0,0.0,0.0,40,5,3,13


In [ ]:
test_merge[['date', 'area'] +
                targets].to_csv("area.csv", index=False)
